## Imports Required

In [0]:
#Install PySyft for federated learning

!pip install syft

In [0]:

# Load libraries to be used

from os import listdir
import numpy as np

import torch
import torch.optim as optim

from torch import nn
from torch.utils.data import Dataset
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models

import syft as sy
from syft.frameworks.torch.federated import FederatedDataset, FederatedDataLoader, BaseDataset

from collections import OrderedDict
import copy
import math
import logging

import warnings 
warnings.filterwarnings("ignore")

In [0]:
# Set up PySyft with 5 virtual workers

hook = sy.TorchHook(torch)
me = hook.local_worker

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
paul = sy.VirtualWorker(hook, id="paul")
john = sy.VirtualWorker(hook, id="john")
ringo = sy.VirtualWorker(hook, id="ringo")
 

In [0]:
bob

## Load the Data

In [0]:
# Mount my Google Drive

from google.colab import drive
drive.mount("/content/gdrive")

In [0]:
!ls "/content/gdrive/My Drive/Colab Notebooks/PetImages"

In [0]:
# Create directory structure
data_dir = '/content/gdrive/My Drive/Colab Notebooks/PetImages'
train_dir = data_dir + '/Train'
valid_dir = data_dir + '/Valid'

In [0]:
# Transformations set up for use with a pretrained library based on Imagenet

transformations = {'train': transforms.Compose([transforms.Resize(224),
                                     transforms.RandomRotation(degrees=15),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])]),
                   'valid': transforms.Compose([transforms.Resize(224),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                          std=[0.229, 0.224, 0.225])])
}

In [0]:
# Create PyTorch dataloaders

dataloaders = {'train' : torch.utils.data.DataLoader(datasets.ImageFolder(train_dir, 
                                           transform=transformations['train']),
                                           batch_size=64, shuffle=True),
               'valid' : torch.utils.data.DataLoader(datasets.ImageFolder(valid_dir, 
                                           transform=transformations['valid']),
                                           batch_size=64, shuffle=True),
              }

In [0]:
# This function for creating a Federated Dataset from a ImageFolder dataset is based on work from 
# https://github.com/andreiliphd/pneumonia-federated-pysyft/blob/master/X_ray_pneumonia-federated-PySyft.ipynb

logger = logging.getLogger(__name__)

def dataset_federate(dataset, workers):
    """
    Add a method to easily transform a torch.Dataset or a sy.BaseDataset
    into a sy.FederatedDataset. The dataset given is split in len(workers)
    part and sent to each workers
    """
    logger.info("Scanning and sending data to {}...".format(", ".join([w.id for w in workers])))
    # take floor to have exactly len(workers) sets after splitting
    
    data_size = math.floor(len(dataset) / len(workers))

    datasets = []
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=data_size, drop_last=True)
    
    for dataset_idx, (data, targets) in enumerate(data_loader):
        worker = workers[dataset_idx % len(workers)]
        logger.debug("Sending data to worker %s", worker.id)
        data = data.send(worker)
        targets = targets.send(worker)
        datasets.append(BaseDataset(data, targets))  # .send(worker)
    logger.debug("Done!")   
    return FederatedDataset(datasets)
  
datasets.ImageFolder.federate = dataset_federate

In [0]:
# Create a federated dataset

federated_dataset = sy.FederatedDataLoader(datasets.ImageFolder(train_dir, 
                                                transformations['train']).federate((bob, alice, paul, john, ringo)),
                                                batch_size=32, shuffle=True)

In [11]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Federated Learning with small CNN

### Creating the Model

In [0]:
class Net(nn.Module):
   
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(7*7*128, 100)
        self.fc2 = nn.Linear(100, 2) 
        
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        ## Define forward behavior
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
        # flatten
        x = x.view(-1, 7*7*128)
        
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)


model = Net()

In [0]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [0]:
log_interval = 32

In [0]:
# Device has been manually set to cpu as error occurring with cuda
device = torch.device("cpu")


### Training the Model

In [0]:
'''
#Training without federated learning

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    
    for batch_idx, data in enumerate(dataloaders['train'],0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if batch_idx % log_interval == 0:    
            print('Epoch: {} Loss: {:.6f}'.format(epoch + 1, running_loss))
            running_loss = 0.0

print('Finished Training')
'''

Epoch: 1 Loss: 0.700743
Epoch: 2 Loss: 0.692102
Finished Training


In [16]:
#Training WITH federated learning

for epoch in range(10): # loop over the dataset multiple times
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(federated_dataset):
        
        model.send(data.location)
        data, target = data.to('cpu'), target.to('cpu')
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        model.get()
        
        if batch_idx % log_interval == 0:
            loss = loss.get() 
            print('Train Epoch: {} Loss: {:.6f}'.format(
                epoch+1, loss.item()))
            
    model.eval()
    
    total_correct = 0
    total = 0
    test_loss = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloaders['valid']):
        
            output = model(data)
            loss = criterion(output, target)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            max_arg_output = torch.argmax(output, dim=1)
        
            total_correct += int(torch.sum(max_arg_output == target))
            total += data.shape[0]
            
            if batch_idx % log_interval == 0:
                print('Validation Loss: {:.6f}'.format(loss.item()))
                print('Accuracy: {:.0%}'.format(total_correct/total))
                print('-'*10)

Train Epoch: 1 Loss: 0.663338
Train Epoch: 1 Loss: 0.686317
Train Epoch: 1 Loss: 0.604885
Validation Loss: 0.741007
Accuracy: 45%
----------
Train Epoch: 2 Loss: 0.951020
Train Epoch: 2 Loss: 0.613803
Train Epoch: 2 Loss: 0.588230
Validation Loss: 0.730383
Accuracy: 48%
----------
Train Epoch: 3 Loss: 0.946745
Train Epoch: 3 Loss: 0.659526
Train Epoch: 3 Loss: 0.593255
Validation Loss: 0.715076
Accuracy: 50%
----------
Train Epoch: 4 Loss: 0.917474
Train Epoch: 4 Loss: 0.664272
Train Epoch: 4 Loss: 0.563322
Validation Loss: 0.714442
Accuracy: 52%
----------
Train Epoch: 5 Loss: 0.975449
Train Epoch: 5 Loss: 0.704608
Train Epoch: 5 Loss: 0.591001
Validation Loss: 0.715202
Accuracy: 50%
----------
Train Epoch: 6 Loss: 0.946001
Train Epoch: 6 Loss: 0.639767
Train Epoch: 6 Loss: 0.560682
Validation Loss: 0.705458
Accuracy: 53%
----------
Train Epoch: 7 Loss: 0.940368
Train Epoch: 7 Loss: 0.660143
Train Epoch: 7 Loss: 0.569180
Validation Loss: 0.719022
Accuracy: 50%
----------
Train Epoch: 

##Federated Learning with Transfer Learning

### Creating the Model

In [17]:
tl_model = models.resnet50(pretrained=True)
tl_model

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:00<00:00, 112883304.93it/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
#Freeze feature parameters so as not to backpropagate through them
for param in tl_model.parameters():
    param.requires_grad = False

In [0]:
#Classifier structure
fc = nn.Sequential(OrderedDict([
                           ('fc1',nn.Linear(2048,102)),
                           ('ReLu1',nn.ReLU()),
                           ('Dropout1',nn.Dropout(p=0.2)),
                           ('fc2',nn.Linear(102,2)),
                           ('output',nn.LogSoftmax(dim=1))
                           ]))

tl_model.fc = fc

In [20]:
tl_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(tl_model.fc.parameters(),lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer,step_size=4,gamma=0.1,last_epoch=-1)
epochs = 2

### Training the Model without Federated Learning

In [0]:
'''
if(train_on_gpu):
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
'''

In [0]:
'''
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    
    model.to('cuda')

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                if(train_on_gpu):
                  inputs, labels = inputs.cuda(), labels.cuda()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    print('Best valid Acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model
    '''

In [0]:
#model_trained = train_model(tl_model, criterion, optimizer, scheduler, epochs)

### Training the Model WITH Federated Learning

In [0]:
model = tl_model

In [0]:
for epoch in range(10): # loop over the dataset multiple times
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(federated_dataset):
        
        model.send(data.location)
        data, target = data.to('cpu'), target.to('cpu')
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        model = user_model.get()
        
        if batch_idx % log_interval == 0:
            loss = loss.get() 
            print('Train Epoch: {} Loss: {:.6f}'.format(
                epoch+1, loss.item()))
            
    model.eval()
    
    total_correct = 0
    total = 0
    test_loss = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloaders['valid']):
        
            output = model(data)
            loss = criterion(output, target)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            max_arg_output = torch.argmax(output, dim=1)
        
            total_correct += int(torch.sum(max_arg_output == target))
            total += data.shape[0]
            
            if batch_idx % log_interval == 0:
                print('Validation Loss: {:.6f}'.format(loss.item()))
                print('Accuracy: {:.0%}'.format(total_correct/total))
                print('-'*10)

## Saving the Model

In [0]:
model_save_name = 'catvdog.pth'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

## Loading the Model